## <center> Market Basket Analysis </center>

- Market basket analysis is one of the applications of association rules. It allows us to predict the products that customers tend to buy in the future by developing a pattern from their past behaviours and habits.
- There are different algorithms for association rule learning one of them is **Apriori Algorithm** and the most suitable product offers will be made for the customer.

## Business Problem
- We will suggest products to users at the basket stage. We are going to apply market basket analysis using the apriori algorithm.

## Importing Libraries

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
!pip install openpyxl

In [3]:
data = pd.read_excel('/kaggle/input/online-retail-dataset/online_retail_II.xlsx', 
                     sheet_name = 'Year 2010-2011')

In [4]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [6]:
data.isna().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [7]:
data.dropna(inplace = True)

In [8]:
data.shape

(406830, 8)

In [9]:
data[['Quantity', 'Price', 'Customer ID']].describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,406830.000,12.061,248.693,-80995.000,2.000,5.000,12.000,80995.000
Price,406830.000,3.461,69.315,0.000,1.250,1.950,3.750,38970.000
Customer ID,406830.000,15287.684,1713.603,12346.000,13953.000,15152.000,16791.000,18287.000


In [10]:
# Removing cancelled transactions.

data_invoice = pd.DataFrame({"Invoice":[row for row in data['Invoice'].values 
                                        if "C" not in str(row)]})
data_invoice.head()
data_invoice = data_invoice.drop_duplicates("Invoice")

In [11]:
# All the transactions except cancelled transactions.
data = data.merge(data_invoice, on = 'Invoice')

In [12]:
# Outlier detection

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    iqr = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * iqr
    low_limit = quartile1 - 1.5 * iqr
    return low_limit, up_limit

# Replace outliers with thresholds
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [13]:
data.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

In [14]:
num_cols = [col for col in data.columns if data[col].dtypes in ["int64", "float64"] and "ID" not in col]
print(num_cols)

['Quantity', 'Price']


In [15]:
for col in num_cols:
    replace_with_thresholds(data, col)

In [16]:
data[['Quantity', 'Price', 'Customer ID']].describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.000,11.834,25.534,1.000,2.000,6.000,12.000,298.500
Price,397925.000,2.893,3.227,0.000,1.250,1.950,3.750,37.060
Customer ID,397925.000,15294.309,1713.173,12346.000,13969.000,15159.000,16795.000,18287.000


In [17]:
data = data[data['Quantity'] > 0]
data = data[data['Price'] > 0]

In [18]:
data.Description.nunique()

3877

In [19]:
data.StockCode.nunique()

3665

- The unique values of the two variables need to be equal because each stock code represents a product.

In [20]:
data_product = data[['Description', 'StockCode']].drop_duplicates()
data_product = data_product.groupby(['Description']).agg({'StockCode':'count'}).reset_index()
data_product.sort_values("StockCode", ascending = False).head()

,Description,StockCode
2014,"METAL SIGN,CUPCAKE SINGLE HOOK",3
1077,EAU DE NILE JEWELLED PHOTOFRAME,2
2460,PINK FAIRY CAKE CUSHION COVER,2
3678,WHITE BAMBOO RIBS LAMPSHADE,2
826,COLOURING PENCILS BROWN TUBE,2


In [21]:
data_product.rename(columns = {"StockCode": "StockCode_Count"}, inplace = True)
data_product = data_product[data_product["StockCode_Count"] > 1]

- Let us delete products with more than one stock code.

In [22]:
data = data[~data['Description'].isin(data_product['Description'])]

In [23]:
print(data.StockCode.nunique())
print(data.Description.nunique())

3630
3858


In [24]:
data_product = data[['Description', 'StockCode']].drop_duplicates()
data_product = data_product.groupby(['StockCode']).agg({'Description':"count"}).reset_index()
data_product.rename(columns = {'Description':"Description_Count"}, inplace = True)

In [25]:
data_product = data_product.sort_values("Description_Count", ascending = False)
data_product.head()

,StockCode,Description_Count
1990,23236,4
1950,23196,4
2826,17107D,3
1886,23131,3
1994,23240,3


In [26]:
data_product = data_product[data_product["Description_Count"] > 1]
data_product.head()

,StockCode,Description_Count
1990,23236,4
1950,23196,4
2826,17107D,3
1886,23131,3
1994,23240,3


In [27]:
data = data[~data["StockCode"].isin(data_product["StockCode"])]

In [28]:
print(data.StockCode.nunique())
print(data.Description.nunique())

3420
3420


- The post statement in the stock code shows the postage cost, let us delete it as it is not a product.

In [29]:
data = data[~data["StockCode"].str.contains("POST", na = False)]

In [30]:
data_germany = data[data['Country'] == "Germany"]
data_germany.shape

(7843, 8)

## Preparing Invoice-Product Matrix for ARL Data Structure

In [31]:
def create_invoice_product(dataframe, id = False):
    if id:
        return dataframe.groupby(['Invoice', 'StockCode'])['Quantity'].sum().unstack(). \
                                            fillna(0).applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(["Invoice", "Description"])['Quantity'].sum().unstack(). \
                                            fillna(0).applymap(lambda x: 1 if x > 0 else 0)

In [32]:
gr_inv_pro_df = create_invoice_product(data_germany, id = True)
gr_inv_pro_df.head()

StockCode,10002,10125,11001,15034,15036,15039,16008,16011,16014,16016,16045,16054,16218,16235,16236,16237,16238,16259,17003,20652,20658,20665,20668,20674,20675,20676,20677,20679,20681,20682,20684,20685,20696,20702,20704,20705,20707,20711,20712,20713,20716,20717,20718,20719,20723,20724,20726,20727,20728,20749,20750,20751,20752,20754,20755,20761,20766,20767,20772,20774,20794,20796,20828,20846,20897,20902,20903,20961,20963,20971,20972,20973,20974,20975,20977,20978,20979,20981,20982,20983,20984,20992,20997,20998,21002,21014,21015,21025,21034,21035,21039,21041,21042,21051,21054,21055,21056,21058,21059,21061,21062,21068,21078,21080,21084,21086,21088,21090,21094,21096,21108,21110,21114,21115,21116,21121,21122,21123,21124,21125,21126,21128,21135,21136,21137,21154,21155,21156,21158,21159,21164,21165,21169,21194,21196,21197,21202,21204,21205,21206,21207,21208,21209,21210,21211,21212,21213,21215,21216,21217,21218,21219,21220,21221,21224,21226,21231,21238,21239,21240,21242,21244,21245,21246,21249,21250,21251,21257,21258,21259,21262,21264,21272,21277,21284,21289,21291,21292,21293,21311,21314,21318,21320,21328,21329,21340,21354,21355,21358,21360,21371,21374,21380,21381,21383,21389,21392,21394,21395,21397,21398,21399,21401,21402,21403,21408,21411,21415,21424,21426,21427,21428,21429,21430,21439,21441,21446,21447,21452,21456,21458,21462,21467,21471,21479,21481,21484,21485,21494,21495,21497,21498,21499,21500,21503,21504,21506,21508,21509,21519,21523,21524,21527,21531,21533,21534,21535,21537,21539,21544,21555,21556,21558,21559,21561,21562,21563,21564,21576,21577,21578,21579,21580,21581,21588,21592,21615,21620,21621,21622,21624,21625,21628,21642,21650,21656,21658,21666,21668,21669,21670,21671,21672,21673,21675,21679,21680,21682,21683,21698,21700,21703,21704,21705,21706,21707,21708,21710,21715,21716,21718,21719,21723,21724,21731,21735,21739,21741,21746,21747,21749,21754,21755,21756,21770,21773,21774,21775,21786,21787,21788,21789,21791,21809,21817,21826,21828,21829,21832,21843,21844,21871,21873,21874,21877,21878,21880,21881,21882,21883,21884,21889,21890,21891,21892,21894,21896,21903,21905,21906,21907,21908,21914,21915,21916,21917,21918,21925,21926,21927,21929,21930,21931,21932,21933,21934,21935,21936,21937,21944,21948,21949,21955,21967,21972,21973,21974,21975,21976,21977,21980,21981,21982,21983,21984,21985,21986,21987,21988,21989,21990,21991,21992,21993,22021,22024,22025,22026,22027,22028,22029,22030,22031,22032,22035,22037,22040,22041,22042,22044,22045,22046,22047,22048,22049,22050,22051,22055,22057,22059,22060,22061,22062,22063,22064,22066,22067,22070,22072,22073,22074,22075,22076,22077,22078,22079,22080,22081,22082,22083,22085,22086,22087,22088,22089,22090,22091,22093,22094,22097,22098,22099,22100,22110,22111,22112,22113,22114,22117,22119,22123,22124,22128,22130,22131,22138,22139,22141,22142,22144,22147,22148,22149,22150,22151,22163,22164,22168,22169,22171,22173,22174,22175,22176,22178,22180,22181,22186,22187,22189,22190,22191,22192,22193,22195,22196,22200,22201,22202,22203,22204,22207,22208,22209,22210,22211,22212,22215,22220,22221,22222,22223,22228,22230,22231,22232,22233,22236,22242,22243,22244,22245,22247,22252,22254,22255,22258,22260,22262,22263,22264,22271,22272,22273,22274,22276,22277,22278,22279,22280,22281,22296,22300,22301,22302,22303,22304,22307,22311,22315,22316,22317,22318,22319,22320,22321,22322,22324,22325,22326,22327,22328,22329,22331,22332,22333,22334,22336,22337,22338,22339,22343,22344,22345,22346,22348,22349,22350,22352,22354,22355,22356,22360,22362,22363,22364,22365,22366,22367,22371,22372,22374,22375,22376,22377,22378,22379,22380,22381,22382,22384,22385,22386,22389,22390,22393,22396,22398,22399,22400,22402,22403,22409,22411,22413,22414,22417,22418,22419,22420,22422,22423,22425,22426,22427,22429,22430,22431,22432,22433,22434,22435,22436,22437,22438,22440,22441,22445,22446,22447,22448,22449,22450,22451,22452,22453,22457,22467,22468,22469,22471,22472,22473,22474,22475,22476,22480,22482,22483,22487,22489,22491,22492,22494,22498,

In [33]:
def check_id(dataframe, stockcode):
    product_name = dataframe[dataframe['StockCode'] == stockcode]['Description'].unique()[0]
    return stockcode, product_name

In [34]:
check_id(data_germany, 10002)

(10002, 'INFLATABLE POLITICAL GLOBE ')

In [35]:
check_id(data_germany, 47480)

(47480, 'HANGING PHOTO CLIP ROPE LADDER')

In [36]:
check_id(data_germany, 22977)

(22977, 'DOLLY GIRL CHILDRENS EGG CUP')

## Determination of Association Rules

- Calculate the support values for every possible configuration of items (threshold of support has been chosen 0.01 (1%))

In [37]:
frequent_itemset = apriori(gr_inv_pro_df, min_support = 0.01, use_colnames = True)

In [38]:
frequent_itemset.head()

,support,itemsets
0,0.013,(10125)
1,0.020,(15036)
2,0.011,(16016)
3,0.016,(16045)
4,0.011,(16235)


In [39]:
rules = association_rules(frequent_itemset, metric = 'support', min_threshold = 0.01)

rules.sort_values("support", ascending = False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1862,(22328),(22326),0.162,0.252,0.135,0.833,3.311,0.094,4.490,0.833
1863,(22326),(22328),0.252,0.162,0.135,0.536,3.311,0.094,1.805,0.933
1913,(22326),(22554),0.252,0.142,0.076,0.304,2.144,0.041,1.233,0.713
1912,(22554),(22326),0.142,0.252,0.076,0.540,2.144,0.041,1.626,0.622
1931,(22326),(22629),0.252,0.106,0.072,0.286,2.705,0.045,1.252,0.842


#### Metrics Explained:
- **antecedent support:** If X is called antecedent, 'antecedent support' computes the proportion of transactions that contain the antecedent X.
- **consequent support:** If Y is called consequent, 'consequent support' computes the proportion of transactions that contain the antecedent Y.
- **support:** 'support' computes the proportion of transactions that contain the antecedent X and Y.
- **confidence:** Probability of buying Y when X is bought.
- **lift:** Represents how many times the probability of getting Y increases when X is received.

Let us sort the dataframe by lift.

In [40]:
sorted_rules = rules.sort_values("lift", ascending=False)

## Suggesting a Product to Users at the Basket Stage

We can develop different strategies at the product offer stage.

For example: when X is bought we can sort according to the probability of buying Y (confidence) and make a product offer, or we can make a offer according to how many times the probability of sales over the lift increases. We can also make a product recommendation with a hybrid filtering where support, lift, and confidence are used together.

In [42]:
product_id = 22554
recommendation_list = []

for idx, product in enumerate(sorted_rules['antecedents']):
    for j in list(product):
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[idx]['consequents'])[0])
            recommendation_list = list(dict.fromkeys(recommendation_list))

**Top 5 most preferred products together with the product with id 22492**

In [43]:
list_top5 = recommendation_list[:5]
list_top5

[21086, 21094, 22728, 21731, 21672]

In [44]:
# Names of the top 5 products

for elem in list_top5:
    print(check_id(data_germany, elem))

(21086, 'SET/6 RED SPOTTY PAPER CUPS')
(21094, 'SET/6 RED SPOTTY PAPER PLATES')
(22728, 'ALARM CLOCK BAKELIKE PINK')
(21731, 'RED TOADSTOOL LED NIGHT LIGHT')
(21672, 'WHITE SPOT RED CERAMIC DRAWER KNOB')


## Putting it all together

In [45]:
def data_filter(dataframe, country = False, Country = ""):
    if country:
        dataframe = dataframe[dataframe['Country'] == Country]
    return dataframe

In [66]:
# Outlier detection

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    iqr = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * iqr
    low_limit = quartile1 - 1.5 * iqr
    return low_limit, up_limit

# Replace outliers with thresholds
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    
def data_prep(dataframe):
    dataframe.dropna(inplace=True)
    
    dataframe = dataframe[~dataframe["StockCode"].str.contains("POST", na = False)]
    dataframe = dataframe[~dataframe["Invoice"].astype(str).str.contains("C", na = False)]
    
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe['Price'] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    
    return dataframe

def create_invoice_product(dataframe, id = False):
    if id:
        return dataframe.groupby(['Invoice', 'StockCode'])['Quantity'].sum().unstack(). \
                                            fillna(0).applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(["Invoice", "Description"])['Quantity'].sum().unstack(). \
                                            fillna(0).applymap(lambda x: 1 if x > 0 else 0)

def check_id(dataframe, stockcode):
    product_name = dataframe[dataframe['StockCode'] == stockcode]['Description'].unique()[0]
    return stockcode, product_name

def apriori_alg(dataframe, support_val = 0.01):
    inv_pro_df = create_invoice_product(dataframe, id = True)
    frequent_itemset = apriori(gr_inv_pro_df, min_support = 0.01, use_colnames = True)
    rules = association_rules(frequent_itemset, metric = 'support', 
                              min_threshold = support_val)

    rules.sort_values("support", ascending = False).head()
    return sorted_rules

def recommend_product(dataframe, product_id, support_val = 0.01, num_of_products = 5):
    sorted_rules = apriori_alg(dataframe, support_val)
    recommendation_list = []
    for idx, product in enumerate(sorted_rules['antecedents']):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[idx]['consequents'])[0])
                recommendation_list = list(dict.fromkeys(recommendation_list))
    return (recommendation_list[:num_of_products])

In [59]:
data = data_prep(data)
data = data_filter(data, country = True, Country = "Germany")
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1098,536527,22809,SET OF 6 T-LIGHTS SANTA,6.000,2010-12-01 13:04:00,2.950,12662.000,Germany
1099,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6.000,2010-12-01 13:04:00,2.550,12662.000,Germany
1100,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12.000,2010-12-01 13:04:00,0.850,12662.000,Germany
1101,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12.000,2010-12-01 13:04:00,1.650,12662.000,Germany
1102,536527,22244,3 HOOK HANGER MAGIC GARDEN,12.000,2010-12-01 13:04:00,1.950,12662.000,Germany


In [69]:
def recommendation_system(dataframe, support_val = 0.01, num_of_products = 5):
    product_id = input("Enter a product id: ")
    
    if product_id in list(dataframe['StockCode'].astype(str).unique()):
        product_list = recommend_product(dataframe, int(product_id), support_val, num_of_products)
        if len(product_list) == 0:
            print("There is no product recommendation.")
        else:
            print("Related products with product id: ", product_id, "\nYou can see them below:")
            
            for i in range(0, len(product_list[:num_of_products])):
                print(check_id(dataframe, product_list[i]))
    else:
        print("Invalid product. Try again!")

In [68]:
recommendation_system(data)

Enter a product id:  22326


Related products with product id:  22326 You can see them below:
(21915, 'RED  HARMONICA IN BOX ')
(22716, 'CARD CIRCUS PARADE')
(22077, '6 RIBBONS RUSTIC CHARM')
(22328, 'ROUND SNACK BOXES SET OF 4 FRUITS ')
(22467, 'GUMBALL COAT RACK')


In [71]:
recommendation_system(data)

Enter a product id:  22725


Related products with product id:  22725 
You can see them below:
(22729, 'ALARM CLOCK BAKELIKE ORANGE')
(22727, 'ALARM CLOCK BAKELIKE RED ')
(22726, 'ALARM CLOCK BAKELIKE GREEN')
(22730, 'ALARM CLOCK BAKELIKE IVORY')
(22728, 'ALARM CLOCK BAKELIKE PINK')


In [72]:
recommendation_system(data)

Enter a product id:  22809


There is no product recommendation.


In [73]:
recommendation_system(data)

Enter a product id:  2256523


Invalid product. Try again!
